In [1]:
from google.colab import drive
drive.mount('/drive')

import requests
import json
import time
import os

from tqdm import tqdm
from datetime import datetime, timedelta

import pandas as pd

Mounted at /drive


In [2]:
path_pag = f'/drive/MyDrive/!SberDataScience/Курсовой проект/парсинг/docs/pagination/'
path_vac = f'/drive/MyDrive/!SberDataScience/Курсовой проект/парсинг/docs/vacancies/'

In [3]:
def getPage(name, area, page = 0):
    """
    Создаем метод для получения страницы со списком вакансий.
    """
    # Справочник для параметров GET-запроса
    params = {
        'search_field':'name',
        'text': name,# Текст фильтра.
        'area': area, # Поиск ощуществляется по вакансиям города Москва
        'page': page, # Индекс страницы поиска на HH
        'per_page': 100 # Кол-во вакансий на 1 странице
    }
     
    req = requests.get('https://api.hh.ru/vacancies', params) # Посылаем запрос к API
    data = req.content.decode() # Декодируем его ответ, чтобы Кириллица отображалась корректно
    req.close()
    return data

# area_dict = {1: 'Мск', 2: 'СПб'} 
area = {113: 'Россия'} 



# names = ['Data Scientist', 'Data Science', 'DataScience', 'Исследователь данных',
#          'Data Analyst', 'Data Analysis', 'Аналитик данных', 'Аналитик',
#          'Data Engineer', 'Data processing', 'Data Engineering',
#          'Python разработчик', 'Python developer', 'Python develop',
#          'Deep Learning', 'Machine Learning', 'AI Researcher', 'AI researcher']

# names = [f'{"Data Scientist"}',f'{"Data Science"}', f'{"DataScience"}', f'{"Исследователь данных"}']
# names = [f'{"Data Analyst"}', f'{"Data Analysis"}', f'{"Аналитик данных"}', f'{"Аналитик"}'] 
# names = [f'{"Data Engineer"}', f'{"Data processing"}', f'{"Data Engineering"}', f'{"Инженер данных"}']
# names = [f'{"ML-инженер"}', f'{"Python разработчик"}', f'{"Python developer"}', f'{"Python develop"}']
# names = [f'{"Deep Learning"}', f'{"Machine Learning"}', f'{"AI Researcher"}', f'{"AI researcher"}']

cat_dict = dict()
cat_dict['Data Science'] = [f'{"Data Scientist"}', f'{"Data Science"}', f'{"ML-инженер"}', f'{"DL-инженер"}', 
                            f'{"Deep Learning"}', f'{"Machine Learning"}', f'{"AI Researcher"}', f'{"AI researcher"}']
cat_dict['Data Analysis'] = [f'{"Data Analyst"}', f'{"Data Analysis"}', f'{"Аналитик данных"}', f'{"Data analytics"}'] 
cat_dict['Data Engineer'] = [f'{"Data Engineer"}', f'{"Data processing"}', f'{"Data Engineering"}', f'{"Инженер данных"}']
cat_dict['Python develop']= [f'{"Python разработчик"}', f'{"Python developer"}', f'{"Python develop"}']


for cat in cat_dict.keys():
  for name in cat_dict[cat]:
    for page in (range(0, 20)):
                
      # Преобразуем текст ответа запроса в справочник Python
      jsObj = json.loads(getPage(name, area, page))
      
      # Сохраняем файлы в папку
      # Определяем количество файлов в папке для сохранения документа с ответом запроса
      # Полученное значение используем для формирования имени документа
      nextFileName = f"{path_pag}/{name}_{len(os.listdir(path_pag))}.json"
                      
      f = open(nextFileName, mode='w', encoding='utf8')
      f.write(json.dumps(jsObj, ensure_ascii=False))
      f.close()
      # Проверка на последнюю страницу, если вакансий меньше 2000
      if (jsObj['pages'] - page) <= 1:
          break
      time.sleep(0.25)
    time.sleep(1)
    print(f'Страницы поиска {cat}:{name} собраны')    
  time.sleep(1) 
  
  

Страницы поиска Data Science:Data Scientist собраны
Страницы поиска Data Science:Data Science собраны
Страницы поиска Data Science:ML-инженер собраны
Страницы поиска Data Science:DL-инженер собраны
Страницы поиска Data Science:Deep Learning собраны
Страницы поиска Data Science:Machine Learning собраны
Страницы поиска Data Science:AI Researcher собраны
Страницы поиска Data Science:AI researcher собраны
Страницы поиска Data Analysis:Data Analyst собраны
Страницы поиска Data Analysis:Data Analysis собраны
Страницы поиска Data Analysis:Аналитик данных собраны
Страницы поиска Data Analysis:Data analytics собраны
Страницы поиска Data Engineer:Data Engineer собраны
Страницы поиска Data Engineer:Data processing собраны
Страницы поиска Data Engineer:Data Engineering собраны
Страницы поиска Data Engineer:Инженер данных собраны
Страницы поиска Python develop:Python разработчик собраны
Страницы поиска Python develop:Python developer собраны
Страницы поиска Python develop:Python develop собраны


In [4]:
count = 1
# Получаем перечень ранее созданных файлов со списком вакансий и проходимся по нему в цикле 
for fl in os.listdir(path_pag):
    
    f = open(f"{path_pag}/{fl}", encoding='utf8')
    jsonText = f.read()
    f.close()
     
    # Преобразуем полученный текст в объект справочника
    jsonObj = json.loads(jsonText)

    # Получаем и проходимся по непосредственно списку вакансий
    for v in tqdm(jsonObj['items']):
        
         
        # Обращаемся к API и получаем детальную информацию по конкретной вакансии
        req = requests.get(v['url'])
        data = req.content.decode()
        req.close()
         
        # Создаем файл в формате json с идентификатором вакансии в качестве названия
        # Записываем в него ответ запроса и закрываем файл
        fileName = f'{path_vac}{v["id"]}.json'
        f = open(fileName, mode='w', encoding='utf8')
        f.write(data)
        f.close()
         
        time.sleep(0.25)
        
    time.sleep(1)  

    print(f"\nСобрано {count*100} из {len(os.listdir(path_pag))*100} вакансий")
    count += 1

print('Вакансии собраны')

100%|██████████| 100/100 [01:26<00:00,  1.16it/s]



Собрано 100 из 6800 вакансий


100%|██████████| 100/100 [01:30<00:00,  1.11it/s]



Собрано 200 из 6800 вакансий


100%|██████████| 74/74 [01:05<00:00,  1.12it/s]


...


Собрано 6600 из 6800 вакансий


100%|██████████| 100/100 [01:16<00:00,  1.31it/s]



Собрано 6700 из 6800 вакансий


100%|██████████| 85/85 [01:06<00:00,  1.28it/s]



Собрано 6800 из 6800 вакансий
Вакансии собраны


In [5]:
# Создаем списки для столбцов таблицы vacancies
IDs = [] # Список идентификаторов вакансий
names = [] # Список наименований вакансий
descriptions = [] # Список описаний вакансий
salary = [] #Факт наличия данных о зарплате
salary_from = []  # Список зарплат 'от'
salary_to = []    # Список зарплат 'до'
salary_currency = [] #Валюта зарплатны
skills_name = [] #Список навыков вакансии
experience = [] #Требуемы опыт
employer = [] #работодатель
schedule = [] #тип занятости
area = []#город
published_at = []#дата публикации
errors = 0
 
cnt_docs = len(os.listdir(f'{path_vac}'))
i = 0
 
# Проходимся по всем файлам в папке vacancies
for fl in tqdm(os.listdir(f'{path_vac}')):
    
    skills_temp = []    

    # Открываем, читаем и закрываем файл
    f = open(f"{path_vac}{fl}", encoding='utf8')
    jsonText = f.read()
    f.close()

    # Текст файла переводим в справочник
    jsonObj = json.loads(jsonText)
    # Заполняем списки для таблиц

    try:
      if jsonObj['errors']:
        errors += 1
        continue
    except:
      pass

    IDs.append(jsonObj['id'])
    names.append(jsonObj['name'])
    descriptions.append(jsonObj['description'])
    experience.append(jsonObj['experience']['name'])
    employer.append(jsonObj['employer']['name'])
    schedule.append(jsonObj['schedule']['name'])
    area.append(jsonObj['area']['name'])
    published_at.append(jsonObj['published_at'])
    
    if jsonObj['salary']:
      salary.append(True)
      salary_from.append(jsonObj['salary']['from'])
      salary_to.append(jsonObj['salary']['to'])
      salary_currency.append(jsonObj['salary']['currency'])
    else:
      salary.append(False)
      salary_from.append(None)
      salary_to.append(None)
      salary_currency.append(None)

    for skl in jsonObj['key_skills']:
        skills_temp.append(skl['name'])
    skills_name_text = ','.join(skills_temp)
    skills_name.append(skills_name_text)


df = pd.DataFrame({'Ids': IDs, 'Employer': employer, 'Name': names, 
                    'Salary': salary, 'From': salary_from, 'To': salary_to, 'Currency': salary_currency,
                    'Experience': experience,
                    'Schedule': schedule,
                    'Skill': skills_name,
                    'Description': descriptions,
                    'Area': area,
                    'Published_at': published_at
                    })



100%|██████████| 2664/2664 [00:04<00:00, 571.75it/s]


In [6]:
errors

1185

In [7]:
time_save = datetime.strftime(datetime.now() + timedelta(hours = 3), '%d.%m.%Y_%H_%M_%S')
path_result = f'/drive/MyDrive/!SberDataScience/Курсовой проект/парсинг/result_df/{time_save}.csv'
df.to_csv(path_result, index = False)

In [8]:
df = pd.read_csv(path_result)
df.shape

(1479, 13)

In [9]:
df.head()

,Ids,Employer,Name,Salary,From,To,Currency,Experience,Schedule,Skill,Description,Area,Published_at
0,49598346,MNOGOGAMES,Аналитик данных / Data scientist,True,130000.0,180000.0,RUR,От 3 до 6 лет,Удаленная работа,"Анализ данных,Python,Аналитическое мышление,pa...",<p><strong>Go Games</strong> - успешная команд...,Рязань,2022-03-20T16:05:38+0300
1,53560338,OVISION,Data Scientist,True,300000.0,NaN,RUR,От 3 до 6 лет,Полный день,"PyTorch,Python,Tensorflow,Computer Vision,OpenCV",<p>Я основатель компании OVISION. В отдел ML м...,Санкт-Петербург,2022-03-21T12:09:03+0300
2,49423067,Точка,Data Scientist,True,NaN,390000.0,RUR,От 3 до 6 лет,Полный день,"аналитика,SQL,Python,Информационные технологии...","<p>Ищем специалиста по Data Science, чтобы стр...",Москва,2022-03-20T12:37:57+0300
3,54057940,DIGINETICA,Data Scientist/ML Engineer (Middle+),True,170000.0,240000.0,RUR,От 1 года до 3 лет,Удаленная работа,"SCALA,Java,SQL,Анализ данных,Python,Elasticsea...",<p><strong>AnyQuery </strong>— SaaS сервис AI ...,Москва,2022-03-21T17:44:24+0300
4,54090749,Дарья Самсонова,CV Data Scientist,True,200000.0,NaN,RUR,От 1 года до 3 лет,Удаленная работа,"Английский язык,Computer Vision,Tensorflow,PyT...",<p><strong>Do[NFT] - </strong>это мультичейн п...,Новосибирск,2022-03-19T18:57:27+0300


In [10]:
df[df['Salary'] == True].count()[0]

259

In [13]:
df1 = pd.read_csv('/drive/MyDrive/!SberDataScience/Курсовой проект/парсинг/result_df/13.03.2022_13_15_16.csv')
df2 = pd.read_csv('/drive/MyDrive/!SberDataScience/Курсовой проект/парсинг/result_df/13.03.2022_15_57_22.csv')
df3 = pd.read_csv('/drive/MyDrive/!SberDataScience/Курсовой проект/парсинг/result_df/13.03.2022_17_55_29.csv')
df4 = pd.read_csv('/drive/MyDrive/!SberDataScience/Курсовой проект/парсинг/result_df/13.03.2022_19_16_54.csv')
df5 = pd.read_csv('/drive/MyDrive/!SberDataScience/Курсовой проект/парсинг/result_df/13.03.2022_19_37_05.csv')
df6 = pd.read_csv('/drive/MyDrive/!SberDataScience/Курсовой проект/парсинг/result_df/16.03.2022_21_34_30.csv')
df7 = pd.read_csv('/drive/MyDrive/!SberDataScience/Курсовой проект/парсинг/result_df/20.03.2022_22_42_07.csv')
df8 = pd.read_csv('/drive/MyDrive/!SberDataScience/Курсовой проект/парсинг/result_df/22.03.2022_12_56_06.csv')
df_result = pd.concat([df1, df2, df3, df4, df5, df6, df7, df8])

In [14]:
df_result.shape

(16507, 13)

In [17]:
df_result.head(3)
df_result.drop_duplicates(inplace = True)
df_result.shape

(11479, 13)

In [18]:
df_result.drop_duplicates(inplace = True)
df_result.shape

df_result.to_csv('/drive/MyDrive/!SberDataScience/Курсовой проект/final_parce.csv')

#Неакутально:Формирование тестового датасета (парсинг df c датами > 19.03)

In [ ]:
# #грузим все что спарсено для тренировочного
# df_parce_train = pd.read_csv('/drive/MyDrive/!SberDataScience/Курсовой проект/final_parce.csv').drop(['Unnamed: 0'], axis = 1)
# print(df_parce_train.shape)
# df_parce_train.head(3)

(10727, 13)


,Ids,Employer,Name,Salary,From,To,Currency,Experience,Schedule,Skill,Description,Area,Published_at
0,49423067,Точка,Data Scientist,True,NaN,320000.0,RUR,От 3 до 6 лет,Полный день,"аналитика,SQL,Python,Информационные технологии...","<p>Ищем специалиста по Data Science, чтобы стр...",Москва,2022-03-11T12:37:57+0300
1,53092406,Астор,Data scientist,True,300000.0,NaN,RUR,От 3 до 6 лет,Полный день,"Python,PyTorch,Machine Learning,Spark,SQL",<strong>Обязанности:</strong> <ul> <li>Работа ...,Москва,2022-03-12T17:01:36+0300
2,53692325,Syndata.io.,Junior Data Scientist (олимпиадные задачи по а...,True,80000.0,120000.0,RUR,Нет опыта,Полный день,"Python,SQL,Английский язык,NLP,ML",<p>частично удаленка</p> <p><strong>Junior-оли...,Москва,2022-03-11T14:28:53+0300


In [ ]:
# #подгружаем новый спарсенный кусок
# df_1 = pd.read_csv('/drive/MyDrive/!SberDataScience/Курсовой проект/парсинг/result_df/20.03.2022_22_42_07.csv')
# print(df_1.shape)
# df_1.head(3)

(2176, 13)


,Ids,Employer,Name,Salary,From,To,Currency,Experience,Schedule,Skill,Description,Area,Published_at
0,49423067,Точка,Data Scientist,True,NaN,390000.0,RUR,От 3 до 6 лет,Полный день,"аналитика,SQL,Python,Информационные технологии...","<p>Ищем специалиста по Data Science, чтобы стр...",Москва,2022-03-20T12:37:57+0300
1,49598346,MNOGOGAMES,Аналитик данных / Data scientist,True,130000.0,180000.0,RUR,От 3 до 6 лет,Удаленная работа,"Анализ данных,Python,Аналитическое мышление,pa...",<p><strong>Go Games</strong> - успешная команд...,Рязань,2022-03-20T16:05:38+0300
2,53560338,OVISION,Data Scientist,True,300000.0,NaN,RUR,От 3 до 6 лет,Полный день,"PyTorch,Python,Tensorflow,Computer Vision,OpenCV",<p>Я основатель компании OVISION. В отдел ML м...,Санкт-Петербург,2022-03-18T12:09:03+0300


In [ ]:
# #сохраняем то, что есть в новом, но нет в старом
# time_save = datetime.strftime(datetime.now() + timedelta(hours = 3), '%d.%m.%Y_%H_%M_%S')
# df_1[~df_1.apply(tuple,1).isin(df_parce_train.apply(tuple,1))].to_csv(f'/drive/MyDrive/!SberDataScience/Курсовой проект/!main/dataset/test/{time_save}.csv', index = False)